### Dependencies

In [1]:
%%shell
# update "apt" which is the local programs on this virtual machine that is Debian Based
sudo apt -y update -q

# install curl, wget, and unzip - these utilities will help us retrieve files online and unzip them as needed.
sudo apt install -y wget curl unzip -q

# Colab has updated from ubuntu 18.04 LTS to 20.04 LTS (LTS stands for "Long-Term Support") Think of google doing this as an upgrade from one Windows version to another.
# To summarize whats below: We are getting the correct chrome driver for the "stable_current" verson of chrome used by colab.
# {
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb
CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
chmod +x /tmp/chromedriver
mv /tmp/chromedriver /usr/local/bin/chromedriver
# }

pip install selenium -q

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:7 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1,063 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:9 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2,803 kB]
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [1,014 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:12 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:13 http://ppa.launchpad.net/graphics-drive

In [2]:
from selenium import webdriver
import pandas as pd
import numpy as np
import os
import time
import requests
from bs4 import BeautifulSoup

# Disable all warnings
import warnings
warnings.filterwarnings("ignore")

### Processing and Cleaning Data

In [4]:
def print_loading_bar(current, total, time, bar_length=40):
    progress = current / total
    completed_length = int(bar_length * progress)
    remaining_length = bar_length - completed_length

    bar = '█' * completed_length + '-' * remaining_length

    print(f"\r[{bar}] Processing {current} of {total} COMPLETE {progress*100:.2f}% Time Lapsed {time:.2f} Seconds", end ="")

def load_join(select, loaded):
    selected_df = pd.read_csv("https://www.dropbox.com/s/" + select + "?dl=1", index_col="review_index")
    if loaded is not None:
        selected_df = pd.merge(loaded, selected_df, on='review_index')
    return selected_df

In [5]:
load_metadata = True
load_ratings = False
load_opinions = False
load_descriptions = False
load_world_happiness_report = True
word_counted_preprocessing_bypass = False

index_df = None
if load_metadata == True:
    load_index = "goqyfyeu2qvpsmo/metadata_df.csv"
    index_df = load_join(load_index, index_df)
if load_ratings == True:
    load_index = "9j2j86xwqrmrljx/ratings_df.csv"
    index_df = load_join(load_index, index_df)
if load_opinions == True:
    load_index = "q8v9f6rbb4z12df/opinion_df.csv"
    index_df = load_join(load_index, index_df)
if load_descriptions == True:
    load_index = "hnp8sebleh6dzgt/descriptions_df.csv"
    index_df = load_join(load_index, index_df)

if load_world_happiness_report == True:
    WHR_df = pd.read_csv("https://www.dropbox.com/s/jyr4e7fleevrb1s/WHR2023.csv?dl=1")

if word_counted_preprocessing_bypass == True:
    joint_df = pd.read_csv("https://www.dropbox.com/s/e64gm79gxu1tgip/joint_df.csv?dl=1")

In [6]:
working_df = index_df.copy()
working_WHR_df = WHR_df.copy()

In [7]:
working_firm_column = working_df['firm'].copy()
working_firm_column.value_counts()

IBM                         60436
McDonald-s                  49450
Deloitte                    46995
EY                          34050
PwC                         33227
                            ...  
i-Net-Solution                  4
Health-Protection-Agency        4
The-Survey-Association          3
NPSA                            2
UKCIL                           1
Name: firm, Length: 428, dtype: int64

In [ ]:
def search_business_industry(business_name):
    url = f"https://www.glassdoor.com/Search/results.htm?keyword={business_name}"

    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-gpu")

    driver = webdriver.Chrome(options=options)

    driver.get(url)
    time.sleep(5)

    html = driver.page_source
    driver.quit()

    return html

html = search_business_industry("EY")
print(html)

In [13]:
with open("test.txt", "w") as file:
    file.write(html)

In [3]:
URL = "https://glassdoor.com"

options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument('--disable-dev-shm-usage')
options.add_argument("--no-sandbox")

driver = webdriver.Chrome(
    options=options
)

driver.get(URL)
title = driver.title
driver.close()

print(title)

Security | Glassdoor


In [ ]:
def search_business_industry(business_name):
    # Set the path to the Chromium browser driver
    chrome_driver_path = "/usr/lib/chromium-browser/chromedriver"

    # Configure the Chrome options
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument(f"--chromedriver-executable={chrome_driver_path}")

    # Create a new Chrome driver instance
    browser = Browser("chrome", options=chrome_options)

    # Construct the search query
    url = f"https://www.glassdoor.com/Search/results.htm?keyword={business_name}"

    # Visit the URL
    browser.visit(url)

    # Wait for the page to load
    time.sleep(2)

    # Get the page source
    html = browser.html

    # Parse the HTML with Beautiful Soup
    soup = BeautifulSoup(html, 'html.parser')

    # Close the browser
    browser.quit()

    return soup

test = search_business_industry("EY")
print(test)

WebDriverException: ignored